<div align="center">

# Обзор научных работ по StyleGAN

</div>

## Методы StyleCLIP
1. **Latent Optimization**:  
    - Оптимизация в пространстве $\mathcal{W}^+$ для минимизации составного CLIP-loss  
    - $\mathcal{W}^+$ состоит из 18 копий базового вектора $\mathcal{W}$ размерности 512
    - Не требует предварительного обучения, но вывод модели (этап оптимизации латента) долгий –  
      в работе указано 98с для NVIDIA GTX 1080Ti на 200-300 итерациях
    - На практике ожидается подбор коэффициентов функции потерь вручную в каждом отдельном случае,  
      т.к. их оптимальный выбор зависит от целевого изображения и характера изменений

2. **Latent Mapper**:  
    - Обучение маппера $M: \mathcal{W}^+ \rightarrow \mathcal{W}^+$
    - Маппер состоит из трёх полносвязных сетей, каждая из которых преобразуют часть набора  
      входных латентов, отвечающую в генераторе за свой уровень деталей изображения
    - В зависимости от задачи и промпта, может быть достаточно обучить только часть маппера
    - Применяется для конкретного типа манипуляции
    - Более быстрый на этапе вывода подход

3. **Global Directions**:  
    - Поиск глобальных направлений в $\mathcal{W}^+$ и в итоге – в $\mathcal{S}$
    - Линейная интерполяция вдоль этих направлений
    - Адаптированная версия этого подхода применяется в StyleGAN-NADA

### Векторы и их значение:
- $t$ (text) - эмбеддинг целевого промпта в текстовом подпространстве CLIP $\mathcal{T}$
- $\Delta t$ - разница между эмбеддингами целевого и исходного описаний
- $i$ (image) - эмбеддинг исходного изображения в подпространстве CLIP для изображений $\mathcal{I}$
- $\Delta i$ - разница между эмбеддингами целевого и исходного изображений в $\mathcal{I}$
- $s$ - "style code", набор параметров слоёв генератора для исходного изображения в стилевом  
    пространстве StyleGAN – $\mathcal{S}$
- $\Delta s$ - искомое изменение этого объекта для изображения в процессе оптимизации

Задачей метода **Global Directions** является отображение вектора $\Delta t$ в $\Delta s$ для создания целевого  
изображения $G(s + \alpha\Delta s)$, где $\alpha$ – коэффициент скорости движения в нужную сторону в стилевом пространстве $\mathcal{S}$.

$\Delta t$ формируется следующим образом:
1. С помощью ImageNet prompt bank формируется определённое кол-во эмбеддингов описания исходного изображения,  
    затем они усредняются для более стабильного результата
2. То же применяется к целевому промпту
3. Нормализованная разница между этими векторами и есть $\Delta t$

</br>

## Метод StyleGAN-NADA

Цель у StyleGAN-NADA та же, что у StyleCLIP – реализовать метод редактирования изображений исключительно  
с помощью текста, используя предобученные модели StyleGAN и CLIP. Основная идея – в дообучении генератора  
StyleGAN, ориентируясь на вывод CLIP в ходе оптимизации.

В начале обсуждения своего метода авторы размышляют о наивной функции потерь, использующей косинусное  
расстояние между эмбеддингами изображения и текста в пространстве CLIP:
$$
\mathcal{L}_{global} = D_{CLIP}\big[G(w), t_{target}\big]
$$
Они приходят в выводу, что такой лосс в чистом виде мало применим в качестве основного в силу склонности  
получаемой модели к артефактам и однородныму выводу (т.н. mode collapse). Однако $D_{CLIP}$ всё же используется  
в работе далее – для выбора обучаемых слоёв генератора.

Далее авторы черпают вдохновение в третьем методе StyleCLIP, который заключается в поиске направления от  
источника к цели в пространстве CLIP. Теперь цель – обучение генератора таким образом, чтобы порождённые  
им изображения отличались от источника исключительно в этом направлении.